# Lab 4.3.6: Model Registry and Version Control

**Module:** 4.3 - MLOps & Experiment Tracking  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand model registry concepts and workflows
- [ ] Register models in MLflow Model Registry
- [ ] Manage model versions and lifecycle stages
- [ ] Implement dataset versioning with hashing
- [ ] Create a complete versioning workflow

---

## 📚 Prerequisites

- Completed: Lab 4.3.5 (Drift Detection)
- Knowledge of: Git basics, MLflow, model deployment concepts
- Hardware: DGX Spark (any configuration)

---

## 🌍 Real-World Context

**"Which model is in production right now?"**

This simple question can cause chaos without proper versioning:

| Scenario | Without Registry | With Registry |
|----------|-----------------|---------------|
| Bug in production | "Which model.pkl is deployed?" 😰 | "Model v3.2 is in Production" ✅ |
| Need rollback | "Where's the old version?" 😰 | "Revert to v3.1" ✅ |
| Audit compliance | "Show your model history" 😰 | Complete lineage available ✅ |
| A/B testing | "How do we track both?" 😰 | v4.0 (Staging) vs v3.2 (Production) ✅ |

**Model Registry is essential for production ML!**

---

## 🧒 ELI5: What is a Model Registry?

> **Imagine you're a baker with multiple cookie recipes.**
>
> Without a recipe book:
> - "Which recipe did we use yesterday?"
> - "I think the secret ingredient was... something?"
> - "Let's just try this random recipe"
>
> With a recipe book (Model Registry):
> - **Recipe v1.0**: Original chocolate chip (testing)
> - **Recipe v1.1**: Added vanilla (staging)
> - **Recipe v2.0**: Double chocolate! (production) ⭐
> - **Recipe v2.1**: With sea salt (archived)
>
> You always know:
> - Which recipe is being used
> - What changed between versions
> - How to roll back if needed
>
> **A Model Registry is your ML recipe book!**

---

## Part 1: Understanding Model Lifecycle

### Model Lifecycle Stages

```
┌─────────────┐    ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│   None      │ -> │   Staging   │ -> │ Production  │ -> │  Archived   │
│ (Testing)   │    │ (Validation)│    │  (Live!)    │    │ (Retired)   │
└─────────────┘    └─────────────┘    └─────────────┘    └─────────────┘
```

| Stage | Purpose | Who Uses It |
|-------|---------|-------------|
| **None** | Initial experiments, testing | Data Scientists |
| **Staging** | Pre-production validation | ML Engineers, QA |
| **Production** | Live serving | End users, applications |
| **Archived** | Retired versions (kept for audit) | Compliance, historical reference |

In [ ]:
import mlflow
import mlflow.pytorch
from mlflow.tracking import MlflowClient
from mlflow.models import infer_signature

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import json
import hashlib
from pathlib import Path
from datetime import datetime
from typing import Dict, Any, Optional

print(f"MLflow version: {mlflow.__version__}")
print(f"PyTorch version: {torch.__version__}")

In [ ]:
# Setup directories and MLflow
NOTEBOOK_DIR = Path.cwd()
MODULE_DIR = (NOTEBOOK_DIR / "..").resolve()
MLFLOW_DIR = MODULE_DIR / "mlflow"
MLFLOW_DIR.mkdir(exist_ok=True)

# Set up MLflow tracking
tracking_uri = f"file://{MLFLOW_DIR}"
mlflow.set_tracking_uri(tracking_uri)

# Create MLflow client for registry operations
client = MlflowClient()

print(f"📁 MLflow tracking: {MLFLOW_DIR}")

---

## Part 2: Creating and Registering Models

Let's create a model and register it in the MLflow Model Registry.

In [ ]:
# Create a simple model for demonstration
class SentimentClassifier(nn.Module):
    """Simple sentiment classifier model."""
    
    def __init__(self, vocab_size: int = 10000, embed_dim: int = 128,
                 hidden_dim: int = 256, num_classes: int = 3):
        super().__init__()
        self.config = {
            "vocab_size": vocab_size,
            "embed_dim": embed_dim,
            "hidden_dim": hidden_dim,
            "num_classes": num_classes
        }
        
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )
    
    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.lstm(embedded)
        hidden = torch.cat([hidden[0], hidden[1]], dim=1)
        return self.classifier(hidden)


# Create model instance
model_v1 = SentimentClassifier()
print(f"Model created: {sum(p.numel() for p in model_v1.parameters()):,} parameters")

In [ ]:
# Set up experiment
EXPERIMENT_NAME = "SentimentClassifier-Development"
MODEL_NAME = "SentimentClassifier"

mlflow.set_experiment(EXPERIMENT_NAME)

# Train and register first model version
print("📦 Registering Model Version 1...")

with mlflow.start_run(run_name="v1-baseline") as run:
    # Log training parameters
    mlflow.log_params({
        "vocab_size": 10000,
        "embed_dim": 128,
        "hidden_dim": 256,
        "learning_rate": 1e-3,
        "epochs": 10,
        "version": "1.0.0"
    })
    
    # Log simulated training metrics
    mlflow.log_metrics({
        "train_accuracy": 0.85,
        "val_accuracy": 0.82,
        "train_loss": 0.35,
        "val_loss": 0.42
    })
    
    # Create input example for signature
    sample_input = torch.randint(0, 10000, (1, 128))
    
    # Log and register the model
    model_info = mlflow.pytorch.log_model(
        model_v1,
        artifact_path="model",
        input_example=sample_input.numpy(),
        registered_model_name=MODEL_NAME  # This registers the model!
    )
    
    run_id_v1 = run.info.run_id

print(f"\n✅ Model v1 registered!")
print(f"   Run ID: {run_id_v1}")
print(f"   Model URI: {model_info.model_uri}")

In [ ]:
# Create and register improved model version
print("📦 Registering Model Version 2 (improved)...")

# Create improved model
model_v2 = SentimentClassifier(embed_dim=256, hidden_dim=512)  # Larger model

with mlflow.start_run(run_name="v2-improved") as run:
    mlflow.log_params({
        "vocab_size": 10000,
        "embed_dim": 256,  # Increased
        "hidden_dim": 512,  # Increased
        "learning_rate": 5e-4,
        "epochs": 15,
        "version": "2.0.0",
        "changes": "Increased model capacity"
    })
    
    mlflow.log_metrics({
        "train_accuracy": 0.91,
        "val_accuracy": 0.88,  # Improved!
        "train_loss": 0.22,
        "val_loss": 0.31
    })
    
    sample_input = torch.randint(0, 10000, (1, 128))
    
    mlflow.pytorch.log_model(
        model_v2,
        artifact_path="model",
        input_example=sample_input.numpy(),
        registered_model_name=MODEL_NAME
    )
    
    run_id_v2 = run.info.run_id

print(f"\n✅ Model v2 registered!")
print(f"   Run ID: {run_id_v2}")

In [ ]:
# View registered model versions
print("📋 REGISTERED MODEL VERSIONS")
print("=" * 60)

try:
    versions = client.search_model_versions(f"name='{MODEL_NAME}'")
    
    for v in versions:
        print(f"\n🔷 Version {v.version}")
        print(f"   Run ID: {v.run_id[:8]}...")
        print(f"   Stage: {v.current_stage}")
        print(f"   Created: {datetime.fromtimestamp(v.creation_timestamp/1000)}")
        if v.description:
            print(f"   Description: {v.description}")
            
except Exception as e:
    print(f"Note: {e}")
    print("Model registry operations may require MLflow server mode.")

---

## Part 3: Managing Model Lifecycle

Transition models through stages: None → Staging → Production → Archived

In [ ]:
def transition_model_stage(
    model_name: str,
    version: int,
    stage: str,
    archive_existing: bool = True
):
    """
    Transition a model version to a new stage.
    
    Args:
        model_name: Name of the registered model
        version: Version number to transition
        stage: Target stage (Staging, Production, Archived, None)
        archive_existing: If True, archive existing models in target stage
    """
    try:
        # Transition to new stage
        client.transition_model_version_stage(
            name=model_name,
            version=version,
            stage=stage,
            archive_existing_versions=archive_existing
        )
        print(f"✅ {model_name} v{version} → {stage}")
        
    except Exception as e:
        print(f"⚠️ Stage transition: {e}")
        print("Note: Full registry features require MLflow server mode.")


# Transition v1 to Staging
print("📦 Transitioning models through lifecycle stages...")
print()

transition_model_stage(MODEL_NAME, version=1, stage="Staging")
transition_model_stage(MODEL_NAME, version=2, stage="Production")

In [ ]:
# Update model version descriptions
def update_model_description(model_name: str, version: int, description: str):
    """Update the description of a model version."""
    try:
        client.update_model_version(
            name=model_name,
            version=version,
            description=description
        )
        print(f"✅ Updated description for {model_name} v{version}")
    except Exception as e:
        print(f"⚠️ Update: {e}")


# Add descriptions
update_model_description(
    MODEL_NAME, 1,
    "Baseline model with standard architecture. Val accuracy: 82%"
)

update_model_description(
    MODEL_NAME, 2,
    "Improved model with larger capacity. Val accuracy: 88%. Ready for production."
)

In [ ]:
# Load model by stage (production workflow)
def load_production_model(model_name: str):
    """
    Load the current production model.
    
    Args:
        model_name: Name of the registered model
    
    Returns:
        Loaded model ready for inference
    """
    model_uri = f"models:/{model_name}/Production"
    
    try:
        model = mlflow.pytorch.load_model(model_uri)
        print(f"✅ Loaded production model from: {model_uri}")
        return model
    except Exception as e:
        print(f"⚠️ Could not load production model: {e}")
        print("Using latest version instead...")
        
        # Fallback to latest version
        model_uri = f"models:/{model_name}/latest"
        return mlflow.pytorch.load_model(model_uri)


# Demo: Load production model
try:
    prod_model = load_production_model(MODEL_NAME)
    
    # Test inference
    test_input = torch.randint(0, 10000, (2, 128))
    prod_model.eval()
    with torch.no_grad():
        output = prod_model(test_input)
    print(f"   Inference test: input {test_input.shape} → output {output.shape}")
    
except Exception as e:
    print(f"Note: {e}")

---

## Part 4: Dataset Versioning

Models are only as good as their data. Let's version datasets too!

In [ ]:
from dataclasses import dataclass
from typing import List


@dataclass
class DatasetVersion:
    """Represents a versioned dataset."""
    name: str
    version: str
    hash: str
    num_samples: int
    num_features: int
    created_at: datetime
    description: str = ""
    source_path: str = ""


def compute_dataframe_hash(df: pd.DataFrame) -> str:
    """
    Compute a deterministic hash for a DataFrame.
    
    Args:
        df: Input DataFrame
    
    Returns:
        SHA256 hash of the DataFrame content
    """
    # Convert to bytes in a deterministic way
    content = df.to_csv(index=False).encode('utf-8')
    return hashlib.sha256(content).hexdigest()[:16]


def create_dataset_version(
    df: pd.DataFrame,
    name: str,
    version: str,
    description: str = ""
) -> DatasetVersion:
    """
    Create a versioned dataset record.
    
    Args:
        df: The DataFrame to version
        name: Dataset name
        version: Version string (e.g., "1.0.0")
        description: Optional description
    
    Returns:
        DatasetVersion object
    """
    return DatasetVersion(
        name=name,
        version=version,
        hash=compute_dataframe_hash(df),
        num_samples=len(df),
        num_features=len(df.columns),
        created_at=datetime.now(),
        description=description
    )


print("✅ Dataset versioning functions defined")

In [ ]:
# Create sample datasets
def generate_sentiment_data(n_samples: int, seed: int = 42) -> pd.DataFrame:
    """Generate synthetic sentiment data."""
    np.random.seed(seed)
    
    return pd.DataFrame({
        "text_length": np.random.randint(10, 500, n_samples),
        "word_count": np.random.randint(5, 100, n_samples),
        "sentiment_score": np.random.uniform(-1, 1, n_samples),
        "has_emoji": np.random.choice([0, 1], n_samples),
        "label": np.random.choice(["negative", "neutral", "positive"], n_samples)
    })


# Create versioned datasets
train_v1 = generate_sentiment_data(10000, seed=42)
train_v2 = generate_sentiment_data(15000, seed=123)  # Larger dataset

dataset_v1 = create_dataset_version(
    train_v1, "sentiment-train", "1.0.0",
    "Initial training dataset, 10k samples"
)

dataset_v2 = create_dataset_version(
    train_v2, "sentiment-train", "2.0.0",
    "Expanded training dataset, 15k samples with more diversity"
)

print("📊 DATASET VERSIONS")
print("=" * 60)

for dv in [dataset_v1, dataset_v2]:
    print(f"\n🗂️ {dv.name} v{dv.version}")
    print(f"   Hash: {dv.hash}")
    print(f"   Samples: {dv.num_samples:,}")
    print(f"   Features: {dv.num_features}")
    print(f"   Description: {dv.description}")

In [ ]:
# Log dataset version with model training
def log_dataset_info(dataset_version: DatasetVersion):
    """Log dataset information to MLflow."""
    mlflow.log_params({
        "dataset_name": dataset_version.name,
        "dataset_version": dataset_version.version,
        "dataset_hash": dataset_version.hash,
        "dataset_samples": dataset_version.num_samples,
        "dataset_features": dataset_version.num_features
    })
    
    # Log dataset manifest as artifact
    manifest = {
        "name": dataset_version.name,
        "version": dataset_version.version,
        "hash": dataset_version.hash,
        "num_samples": dataset_version.num_samples,
        "num_features": dataset_version.num_features,
        "created_at": dataset_version.created_at.isoformat(),
        "description": dataset_version.description
    }
    
    manifest_path = "/tmp/dataset_manifest.json"
    with open(manifest_path, 'w') as f:
        json.dump(manifest, f, indent=2)
    
    mlflow.log_artifact(manifest_path, artifact_path="data")


# Demo: Train with dataset versioning
print("📦 Training model with dataset versioning...")

with mlflow.start_run(run_name="v3-with-versioned-data") as run:
    # Log dataset information
    log_dataset_info(dataset_v2)
    
    # Log model parameters
    mlflow.log_params({
        "model_version": "3.0.0",
        "learning_rate": 3e-4,
        "epochs": 20
    })
    
    # Log metrics
    mlflow.log_metrics({
        "train_accuracy": 0.93,
        "val_accuracy": 0.90
    })
    
    # Log and register model
    model_v3 = SentimentClassifier(embed_dim=256, hidden_dim=512)
    sample_input = torch.randint(0, 10000, (1, 128))
    
    mlflow.pytorch.log_model(
        model_v3,
        artifact_path="model",
        input_example=sample_input.numpy(),
        registered_model_name=MODEL_NAME
    )

print(f"\n✅ Model v3 trained with versioned data")
print(f"   Dataset: {dataset_v2.name} v{dataset_v2.version}")
print(f"   Data hash: {dataset_v2.hash}")

---

## Part 5: Complete Versioning Workflow

Let's put it all together in a production-ready workflow.

In [ ]:
class ModelVersioningWorkflow:
    """
    Complete model versioning workflow.
    
    Manages the full lifecycle from training to production.
    """
    
    def __init__(self, model_name: str, experiment_name: str):
        self.model_name = model_name
        self.experiment_name = experiment_name
        self.client = MlflowClient()
        
        mlflow.set_experiment(experiment_name)
    
    def train_and_register(
        self,
        model: nn.Module,
        dataset_version: DatasetVersion,
        metrics: Dict[str, float],
        params: Dict[str, Any],
        description: str = ""
    ) -> str:
        """
        Train and register a new model version.
        
        Returns:
            Model version number
        """
        with mlflow.start_run() as run:
            # Log dataset info
            log_dataset_info(dataset_version)
            
            # Log parameters and metrics
            mlflow.log_params(params)
            mlflow.log_metrics(metrics)
            
            # Log model
            sample_input = torch.randint(0, 10000, (1, 128))
            
            model_info = mlflow.pytorch.log_model(
                model,
                artifact_path="model",
                input_example=sample_input.numpy(),
                registered_model_name=self.model_name
            )
            
            # Get the version number
            versions = self.client.search_model_versions(f"name='{self.model_name}'")
            latest_version = max(int(v.version) for v in versions)
            
            # Update description
            if description:
                self.client.update_model_version(
                    name=self.model_name,
                    version=latest_version,
                    description=description
                )
            
            return str(latest_version)
    
    def promote_to_staging(self, version: int) -> bool:
        """Promote a model version to Staging."""
        try:
            self.client.transition_model_version_stage(
                name=self.model_name,
                version=version,
                stage="Staging"
            )
            print(f"✅ v{version} promoted to Staging")
            return True
        except Exception as e:
            print(f"⚠️ Promotion failed: {e}")
            return False
    
    def promote_to_production(self, version: int, archive_current: bool = True) -> bool:
        """Promote a model version to Production."""
        try:
            self.client.transition_model_version_stage(
                name=self.model_name,
                version=version,
                stage="Production",
                archive_existing_versions=archive_current
            )
            print(f"✅ v{version} promoted to Production")
            return True
        except Exception as e:
            print(f"⚠️ Promotion failed: {e}")
            return False
    
    def rollback(self, target_version: int) -> bool:
        """Roll back to a previous model version."""
        print(f"🔄 Rolling back to v{target_version}...")
        return self.promote_to_production(target_version)
    
    def get_production_model(self):
        """Load the current production model."""
        model_uri = f"models:/{self.model_name}/Production"
        return mlflow.pytorch.load_model(model_uri)
    
    def get_version_history(self) -> List[Dict]:
        """Get the history of all model versions."""
        versions = self.client.search_model_versions(f"name='{self.model_name}'")
        
        history = []
        for v in versions:
            history.append({
                "version": v.version,
                "stage": v.current_stage,
                "created": datetime.fromtimestamp(v.creation_timestamp/1000),
                "description": v.description or "No description"
            })
        
        return sorted(history, key=lambda x: int(x["version"]))


print("✅ ModelVersioningWorkflow class defined")

In [ ]:
# Demo: Complete workflow
print("🔄 COMPLETE VERSIONING WORKFLOW DEMO")
print("=" * 60)

workflow = ModelVersioningWorkflow(
    model_name=MODEL_NAME,
    experiment_name=EXPERIMENT_NAME
)

# Step 1: Train and register a new model
print("\n📦 Step 1: Train and register new model")
new_model = SentimentClassifier(embed_dim=384, hidden_dim=768)

try:
    version = workflow.train_and_register(
        model=new_model,
        dataset_version=dataset_v2,
        metrics={"train_accuracy": 0.94, "val_accuracy": 0.91},
        params={"embed_dim": 384, "hidden_dim": 768, "epochs": 25},
        description="Largest model yet. Best validation accuracy."
    )
    print(f"   New version: {version}")
except Exception as e:
    print(f"   Note: {e}")
    version = "4"

# Step 2: View version history
print("\n📜 Step 2: Version history")
try:
    history = workflow.get_version_history()
    for v in history:
        print(f"   v{v['version']}: {v['stage']:<12} - {v['description'][:40]}")
except Exception as e:
    print(f"   Note: {e}")

---

## ✋ Try It Yourself: Exercise

**Task:** Create a versioning workflow for your use case.

1. Define a model for your domain
2. Create 3 versions with different configurations
3. Register all versions with meaningful descriptions
4. Promote the best version to Production
5. Simulate a rollback scenario

<details>
<summary>💡 Hint</summary>

```python
# Create workflow
workflow = ModelVersioningWorkflow(
    model_name="MyModel",
    experiment_name="MyExperiment"
)

# Train multiple versions
for config in [{"size": "small"}, {"size": "medium"}, {"size": "large"}]:
    model = create_model(config)
    version = workflow.train_and_register(
        model=model,
        dataset_version=dataset,
        metrics={"accuracy": ..., "loss": ...},
        params=config,
        description=f"Model with {config['size']} configuration"
    )

# Promote best
workflow.promote_to_production(version="3")
```
</details>

In [ ]:
# YOUR CODE HERE

# Step 1: Define your model


# Step 2: Create 3 versions


# Step 3: Register with descriptions


# Step 4: Promote to production


# Step 5: Simulate rollback


---

## ⚠️ Common Mistakes

### Mistake 1: Not Tracking Data with Models

In [ ]:
# ❌ WRONG: Only tracking the model
# mlflow.pytorch.log_model(model, "model")
# "What data was this trained on?" 😰

# ✅ RIGHT: Track data and model together
# log_dataset_info(dataset_version)
# mlflow.pytorch.log_model(model, "model")
# Complete lineage! ✅

print("Always log dataset information with your model!")

### Mistake 2: No Clear Promotion Criteria

In [ ]:
# ❌ WRONG: Promoting without criteria
# "This looks good, let's ship it!"

# ✅ RIGHT: Define clear promotion gates
promotion_criteria = {
    "staging": {
        "min_val_accuracy": 0.85,
        "max_val_loss": 0.5,
        "required_tests": ["unit_tests", "integration_tests"]
    },
    "production": {
        "min_val_accuracy": 0.88,
        "staging_time_hours": 24,  # At least 24h in staging
        "a_b_test_passed": True
    }
}

print("Define clear criteria for each promotion stage!")

---

## 🎉 Checkpoint

You've learned:
- ✅ Model registry concepts and lifecycle stages
- ✅ Registering and versioning models in MLflow
- ✅ Managing model transitions (Staging → Production)
- ✅ Dataset versioning with content hashing
- ✅ Building complete versioning workflows

---

## 📖 Further Reading

- [MLflow Model Registry](https://mlflow.org/docs/latest/model-registry.html)
- [DVC for Data Versioning](https://dvc.org/)
- [Hugging Face Model Hub](https://huggingface.co/docs/hub/models)
- [ML Model Management Best Practices](https://neptune.ai/blog/ml-model-management)

---

## 🧹 Cleanup

In [ ]:
import gc

gc.collect()

if torch.cuda.is_available():
    torch.cuda.empty_cache()

print(f"📁 MLflow data saved to: {MLFLOW_DIR}")
print("✅ Resources cleaned up")

---

## 📝 Summary

In this lab, we:

1. **Learned** model registry concepts and lifecycle stages
2. **Registered** multiple model versions in MLflow
3. **Managed** model transitions through stages
4. **Implemented** dataset versioning with content hashing
5. **Built** a complete versioning workflow class

**Next up:** Lab 4.3.7 - Reproducibility Audit!